# 11.2.5 Rank-k Approximation

<font color=red> Be sure to make a copy!!!! </font>

This notebook walks you through the operations required to compute a low rank approximation of a matrix $ B $. We will create a matrix $A$ whose column space will be used in the approximation of $B$.

We start by creating a random $ m \times n $ matrix $ B $.  We then take $ k $ columns of $ B $ to be matrix $ A $, whose columns will be used in the approximation $ B \approx A V $.
(In the text and the videos, we talk about computing $ W $ so that $ B \approx A W^T $.  Here we find it more convenient to compute the transpose of that matrix instead.  We call it $ V $ to distinguish it from $ W $.  So, $ W = V^T $.)

$ V $ is computed as $ ( A^T A )^{-1} A^T B $. 

In [14]:
import numpy as np
import laff
import flame

m = 8
n = 8
k = 3

# Random matrix of size mxn
B = np.matrix( np.random.rand( m, n ) )

# A is k columns of B taken at even intervals
if 2*k <= n: #k is less than half of n
    interval = np.ceil( n/k ) 
    A = B[ :, ::int(interval) ] # These are slices in python. 
                           # This says take all rows of B, and columns 
                           # from 0 to the end at interval steps
else:
    A = B[ :, :k] #If k is greater than half of n, then just take the first k columns

print( 'A = ' )
print( A )

print( 'B = ' )
print( B )


A = 
[[0.84541453 0.95411398 0.70184806]
 [0.49813656 0.37799136 0.98955358]
 [0.09853289 0.91297539 0.42485301]
 [0.2058494  0.68039421 0.19806898]
 [0.0448449  0.06870541 0.41366457]
 [0.2911061  0.10433575 0.86866266]
 [0.55104708 0.91130121 0.49892453]
 [0.85050982 0.20707356 0.31460101]]
B = 
[[0.84541453 0.85125907 0.96614224 0.95411398 0.15654782 0.06051364
  0.70184806 0.14405197]
 [0.49813656 0.57741384 0.89517515 0.37799136 0.92982326 0.86678317
  0.98955358 0.05057198]
 [0.09853289 0.33311593 0.04525893 0.91297539 0.73826441 0.09809916
  0.42485301 0.83455429]
 [0.2058494  0.70886967 0.6679021  0.68039421 0.72092874 0.17051832
  0.19806898 0.13052371]
 [0.0448449  0.8700187  0.39528943 0.06870541 0.79121594 0.26853585
  0.41366457 0.13383882]
 [0.2911061  0.72235435 0.77372904 0.10433575 0.02756425 0.97355349
  0.86866266 0.05804332]
 [0.55104708 0.32423259 0.62744237 0.91130121 0.37670364 0.30472132
  0.49892453 0.79964887]
 [0.85050982 0.05170656 0.98373876 0.20707356 0.78

We start the process of computing $( A^T A )^{-1} A^T B$ by computing $ A^T A $ and storing the result in a matrix, $C$.  In this implementation, we ignore symmetry.

In [15]:
C = np.transpose( A ) * A 

print( 'C = ' )
print( C )

C = 
[[2.12872212 1.9366718  1.98284564]
 [1.9366718  3.23862704 2.20520073]
 [1.98284564 2.20520073 2.96513107]]


Now, form $ V = A^T B $, notice that we are not done forming $V$ after this step.

In [16]:
V = np.transpose( A ) * B

Instead of computing $ C^{-1} = ( A^T A )^{-1} $ explicitly, we notice that we can instead store the $ L $ and $ U $ factorization of $C$ in $ C $ and then just solve $ L ( U X ) = V $. First we will overwrite $ V $ with the 
result of solving $ L Z = V $, and then we will overwrite $ V $ with the result of solving $ U X = V $.

Copy your `LU_unb_var5` routine from *Notebook 6.3: Solving A x b via LU Factorization and Triangular Solves*

In [17]:
import flame
import laff

def LU_unb_var5(A):

    ATL, ATR, \
    ABL, ABR  = flame.part_2x2(A, \
                               0, 0, 'TL')

    while ATL.shape[0] < A.shape[0]:

        A00,  a01,     A02,  \
        a10t, alpha11, a12t, \
        A20,  a21,     A22   = flame.repart_2x2_to_3x3(ATL, ATR, \
                                                       ABL, ABR, \
                                                       1, 1, 'BR')

        #------------------------------------------------------------#

        laff.invscal( alpha11, a21 )        #  a21 := a21 / alpha11
        laff.ger( -1.0, a21, a12t, A22 )    #  A22 := A22 - a21 * a12t

        #------------------------------------------------------------#

        ATL, ATR, \
        ABL, ABR  = flame.cont_with_3x3_to_2x2(A00,  a01,     A02,  \
                                               a10t, alpha11, a12t, \
                                               A20,  a21,     A22,  \
                                               'TL')

    flame.merge_2x2(ATL, ATR, \
                    ABL, ABR, A)

Now run `LU_unb_var5` on the matrix $C$ to store $L$ and $U$ in it.

In [18]:
LU_unb_var5( C )

In [20]:
C

matrix([[2.12872212, 1.9366718 , 1.98284564],
        [0.90978141, 1.47667904, 0.40124463],
        [0.93147228, 0.27172095, 1.00913875]])

In [23]:
V

matrix([[2.12872212, 1.65798301, 2.83005061, 1.9366718 , 1.73319775,
         1.76199877, 1.98284564, 1.50543798],
        [1.9366718 , 2.25821608, 2.6393167 , 3.23862704, 2.22790381,
         1.17637286, 2.20520073, 1.94258893],
        [1.98284564, 2.61617955, 3.17358627, 2.20520073, 2.27176219,
         2.36962305, 2.96513107, 1.32698336]])

Solve $L ( U X ) = V$, overwriting $V$ where $U$ and $L$ are stored in the upper and the strictly lower portions of $C$ respectively.

In [27]:
laff.trsm('Left','Lower triangular', 'No transpose', 'Unit diagonal', C, V)

In [28]:
C

matrix([[2.12872212, 1.9366718 , 1.98284564],
        [0.90978141, 1.47667904, 0.40124463],
        [0.93147228, 0.27172095, 1.00913875]])

In [29]:
V

matrix([[ 1.00000000e+00, -2.71711150e-01,  9.37123267e-01,
         -5.12388393e-17,  8.73084535e-02,  5.18096227e-01,
          0.00000000e+00,  5.10815545e-01],
        [-9.09781406e-01,  5.21230322e-01, -9.48832186e-01,
          1.00000000e+00,  2.32111995e-01, -9.87624174e-01,
          0.00000000e+00, -1.45250660e-02],
        [-6.84265612e-01,  9.71675104e-01, -9.98728532e-02,
         -2.71720949e-01,  3.31680594e-01,  6.22422636e-01,
          1.00000000e+00, -7.00750207e-01]])

In [31]:
laff.trsm('Left','Upper triangular', 'No transpose', 'Nonunit diagonal', C, V)

In [32]:
C

matrix([[2.12872212, 1.9366718 , 1.98284564],
        [0.90978141, 1.47667904, 0.40124463],
        [0.93147228, 0.27172095, 1.00913875]])

In [33]:
V

matrix([[ 1.49426062, -1.10763298,  1.09252392, -0.4318541 , -0.32689192,
          0.42981313, -0.67806891,  0.72406928],
        [-0.4318541 ,  0.09134121, -0.61565284,  0.75035888,  0.06787674,
         -0.83640805, -0.26926025,  0.17884787],
        [-0.67806891,  0.96287562, -0.09896841, -0.26926025,  0.3286769 ,
          0.61678598,  0.99094401, -0.69440422]])

The $ j $th column of $ A V $ now equals the projection of the $ j $th column of $ B $ onto the column space of $ A $, $ {\cal C}( A ) $. 

A couple of notes:
    
-    The matrix $ A^T A $ is symmetric positive definite.  As a result, one does not need to pivot when performing LU factorization.  (The reason for this is beyond the scope of this course.)

-    One could use what is called a "Symmetric rank-k update" operation to compute only the lower (or upper) triangular part of $ A^T A $.  This would (approximately) halve the number of floating point operations that are required.

-    In one of the enrichments, 8.4.2, we discussed the Cholesky factorization of a symmetric positive definite matrix.   One should ideally use that here, since it also takes advantage of symmetry.

- This would then leave us with $ L $, a lower triangular matrix, such that $ C = A^T A = L L^T $.  Computing $ V $ would then require the steps
  - $ V = A^T B $.
  - Solve $ L Z = V $ overwriting $ V $ with $ Z $.
  - Solve $ L^T X = V $ overwriting $ V $ with $ X $.
    

## A routine

The above computation should be implemented as the routine <code> RankKApprox( B, k ) </code>
where $ B $ is the $ m \times n $ matrix to be approximated, and $k$ is the rank of the eventual approximation that will be returned by the method. 

In [39]:
def RankKApprox( B, k ):
    m,n = B.shape # How many rows and columns does B have?

    # A is k columns of B taken at even intervals
    if 2*k <= n: #k is less than half of n
        interval = np.ceil( n/k ) 
        A = B[ :, ::int(interval) ] # These are slices in python. 
                               # This says take all rows of B, and columns 
                               # from 0 to the end at interval steps
    else:
        A = B[ :, :k] #If k is greater than half of n, then just take the first k columns
    
    # C = A^T A
    C = np.transpose( A ) * A   
    # W = A^T B
    W = np.transpose( A ) * B
    # Overwrite C with its LU factorization
    LU_unb_var5( C )
    
    # Solve L(UX) = W, overwriting W with X
    laff.trsm('Left','Lower triangular', 'No transpose', 'Unit diagonal', C, W)
    laff.trsm('Left','Upper triangular', 'No transpose', 'Nonunit diagonal', C, W)
    
    return A * W

## An Application: Rank k Image Approximation

Now that we have implemented routines to create low rank approximations to matrices we will explore what a rank k approximation to an image looks like. Each pixel in an image can be thought of as a value in a matrix. For a grayscale image, this value corresponds to how black or white it is on a relative scale.

We will use two techniques for these approximations. First, the normal approximation developed above and second, the SVD which is a very useful matrix decomposition that guarantees the best approximation given $k$ columns. The SVD might take a while to compute, so don't panic if one of the code blocks takes a bit to complete.

Try experimenting with the number of columns below by changing the `numCols` variable.

If you want to use your own images, make sure that they are in the `png` format and just place them in your notebooks directoy. Then change the `filename` variable to reflect the file name of your image.

In [40]:
%matplotlib

Using matplotlib backend: QtAgg


In [52]:
from im_approx import *

# Try varying the number of columns used for the approximation 
numCols = 40

# Load an image
filename = 'building.png'

img = np.matrix(read_image( filename ))

In [53]:
# Make the approximations using normal equations and the SVD
# This step might take a while as it is a lot of computation.
normalApprox, SVDApprox = create_approximations( img, k=numCols, approximator=RankKApprox )

In [60]:
#Now plot the approximations that we have created

# Note that we're having some issues with our 
# approximations being somewhat darker than the 
# real image and are investigating.
plot_approximations( img, normalApprox, SVDApprox, k=numCols )

![](Figure_1.png)

## Hints for implementing RankKApprox

<font color=blue> # C = A^T A: </font> This comment corresponds to computing $C = A^T A$. Try using numpy's built in transpose method by calling `np.transpse(A)`. 

<font color=blue># W = A^T B: </font> This comment corresponds to computing $W = A^T B$. See above for a hint.

<font color=blue># Overwrite C with its LU factorization: </font> Use your implementation of LU_unb_var5 from an earlier notebook for this.
    
<font color=blue># Solve L(UX) = W, overwriting W with X: </font> Use `laff.trsm` to do this. Recall that "trsm" means triangular solve with multiple right hand sides.